# **REPORTE DE AJUSTES MANUALES FALABELLA.COM**

### **Usuario**


In [1]:
#IMPORTANTE: 
#Solo se debe modificar el usuario, ruta (en caso sea diferente) y fechas de reporte. 
#Considerar que se debe sincronizar la carpeta 'MARKETPLACE PERÚ' antes de realizar la ejecución del script.

usuario = r'gfloress'
ruta = r'MARKETPLACE PERÚ - Documents'

year = '2023'  # Año
week = '43' # Semana
fecha_corte = '2023-10-25' # Fecha Lunes

# hola!!!

In [2]:
import pandas as pd
import numpy as np

C:\Users\gfloress\AppData\Local\Temp\ipykernel_9092\2162656668.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
x = 2
x + 1

3

#### **Librerías**

In [777]:
import os
import glob 
import shutil
import warnings

import numpy as np
import pandas as pd

warnings.filterwarnings(action='ignore')

pd.set_option('display.expand_frame_repr', False)
pd.options.display.float_format = '{:.2f}'.format


In [778]:
# Fechas
fecha_corte = pd.to_datetime(fecha_corte)
fecha_corte_txt = fecha_corte.strftime('%d_%m_%Y')

### **Rutas**

In [779]:
# Rutas
ruta_base = r'C:\\Users\\' + usuario + r'\\Falabella\\' + ruta
ruta_postventa_fcom = ruta_base + r'\\1. Postventa\\3. F.com'
ruta_ajustes = os.path.join(ruta_postventa_fcom, "1. Ajustes", year, "Ajustes manuales", "Semana_"+week+"_Ajustes_"+fecha_corte_txt)

# Ruta temporal
temp_dir = r'C:\\Users\\' + usuario + r'\\OneDrive - Falabella\\Escritorio\\temp'
os.makedirs(temp_dir, exist_ok=True)

In [780]:
cmr_files = glob.glob(os.path.join(ruta_ajustes, "Forms Recibidos", "CMR", "[A-Z]*.xlsx"))
comisiones_files = glob.glob(os.path.join(ruta_ajustes, "Forms Recibidos", "SC*.xlsx"))
siniestros_files = glob.glob(os.path.join(ruta_ajustes, "Forms Recibidos", "Plantilla*.xlsx"))

### **Variables**

### **Funciones**

In [781]:
# Moves files to temporary folder
def process_file(file_path):
    # Normalize the file path to ensure correct path separators
    file_path = os.path.normpath(file_path)

    # Original long file path
    original_path = file_path

    # Generate the temporary path based on the file's name
    file_name = os.path.basename(original_path)
    temp_path = os.path.join(temp_dir, file_name)

    try:
        # Copy the file to the temporary path
        shutil.copy2(original_path, temp_path)

        # Copy the file back to the original location
        shutil.copy2(temp_path, original_path)
    except Exception as e:
        print(f"An error occurred while processing file {file_path}: {e}")

In [782]:
# Delete temporary files from temporary folder
def delete_temp_files(temp_dir):
    try:
        for file_name in os.listdir(temp_dir):
            file_path = os.path.join(temp_dir, file_name)
            if os.path.isfile(file_path):
                os.remove(file_path)
    except Exception as e:
        pass

In [783]:
# Read data files from temporary folder and delete files from this folder
def process_data_files(files, temp_dir):
    for file in files:
        process_file(file)

    data = pd.DataFrame()
    x = pd.DataFrame()

    files = glob.glob(os.path.join(temp_dir, "[A-Z]*.xlsx"))

    for file in files:
        try:
            x = pd.read_excel(file, 'Order Item Transactions')
        except:
            try:
                x = pd.read_excel(file, 'Hoja1')
            except:
                try:
                    x = pd.read_excel(file, 'plantilla comision')
                except:
                    try:
                        x = pd.read_excel(file, 'plantilla cmr')
                    except:
                        try:
                            x = pd.read_excel(file, 'Query result')
                        except:
                            x = pd.DataFrame()

        x['File'] = file
        data = data.append(x)
    
    delete_temp_files(temp_dir)

    data['File'] = data['File'].str.extract(r'temp\\(.*)')

    return data

In [784]:
# Convert data types
def convert_data_types(data):

    num_cols = ['Amount*']
    data[num_cols] = data[num_cols].apply(lambda x: pd.to_numeric(x.astype(str).str.replace(',', '.'), errors='coerce', downcast='float'))

    text_cols = ['Falabella SKU','Order Item Id','Delivery Order Number']
    data[text_cols] = data[text_cols].apply(lambda x: x.astype(str).str.replace('(\.0$)', ''))

    date_cols = ['Fecha de envío']
    data[date_cols] = data[date_cols].apply(lambda x: pd.to_datetime(x, errors='coerce').dt.strftime('%d/%m/%Y'))

    return data

In [785]:
# Validate transactions
def format_validation(data, validacion_linea=True, validacion_siniestro=False):

    # 1: Validación por líneas para cada 'Delivery Order Number' y 'Order Item Id'
    
    if validacion_linea:
        val1 = (
            data 
            .groupby(['Delivery Order Number','Order Item Id','Falabella SKU','Account Statement Group*'], dropna=False) 
            .agg(n=('Account Statement Group*', 'count')) 
            .reset_index() 
            .pivot_table(
                index=['Delivery Order Number','Order Item Id','Falabella SKU'],
                columns=['Account Statement Group*'],
                values='n'
            ) 
            .reset_index() 
            .assign(**{
                'validacion_linea': lambda x: np.where(
                    (x['Commission Adjustment']==1) & (x['Commission Credit Adjustment']==1), 'Si', 'No'
                )
            }) 
        )

        df = (
            data
            .merge(
                val1.loc[:,['Delivery Order Number','Order Item Id','validacion_linea']], 
                on=['Delivery Order Number','Order Item Id'],
                how='left'
                )
        )
    else:
        df = (
            data
            .assign(**{'validacion_linea': 'Si'})
        )

    # 2: Validación de comentarios 'Account Statement Group*'

    if validacion_linea:
        df = (
            df
            .assign(**{
                'Comment': 
                    lambda x: np.select([
                                    # añadir lógicas
                                    x['Account Statement Group*']=='Commission Credit Adjustment',
                                    x['Account Statement Group*']=='Commission Adjustment'
                                ], [
                                    'Reintegro a favor del seller por descuento CMR',
                                    #'Reintegro a favor del seller por descuento CMR + Comisión negociada',
                                    'Comisión aplicada al reintegro a favor de Falabella por descuento CMR'
                                ], 
                                default=''
                                )
            })
        )
    else:
        if validacion_siniestro:
            df = (
                df
                .assign(**{'Account Statement Group*': lambda x: 'Lost or Damaged (Order Item Level) Credit'})
            )
        else:
            df = df #(
                #df
                #.assign(**{'Account Statement Group*': lambda x: 'Commision Credit Adjustment'})
            #)

    # 3. Validación de códigos 'SellerId'

    df = (
        df
        .assign(**{
            'validacion_seller_len': lambda x: np.where(
                ((x['Seller ID*'].notnull()) | (x['Seller ID*']==7) | (x['Seller ID*'].str.contains('^SC', na=False))), 'Si', 'No'
            )
        })
    )

    return df

In [806]:
cmr.query("~`Seller ID*`.str.contains('^SC', na=False)")

,Revisar comentario,Fecha de envío,A Favor de,Falabella SKU,Order Item Id,Solicitante,Area,categoría,Nombre del Seller en FSC,Seller ID*,Account Statement Group*,Seller Order Line Id*,Delivery Order Number,Amount*,VAT,WHT,Currency*,Comment,NÚMERO DE FACTURA,CENTRO DE COSTOS,validacion_linea,validacion_seller_len,validacion_devolucion,validacion_ajuste,validacion_seller_id,validacion_monto_porc,validacion_monto_alto
668,NaN,16/10/2023,A Favor del Seller,120800557,11780563,Alexis Ibañez,Comercial,AUDIO,OFERTEC PERU,NaN,Commission Credit Adjustment,NaN,2100315405,92.48,NaN,NaN,PEN,Reintegro a favor del seller por descuento CMR,NaN,NaN,Si,No,Si,Si,No,No,Si
669,NaN,16/10/2023,A Favor del Seller,120800557,11692250,Alexis Ibañez,Comercial,AUDIO,OFERTEC PERU,NaN,Commission Credit Adjustment,NaN,2099465875,92.48,NaN,NaN,PEN,Reintegro a favor del seller por descuento CMR,NaN,NaN,Si,No,Si,Si,No,No,Si
670,NaN,16/10/2023,A Favor del Seller,120800557,11595034,Alexis Ibañez,Comercial,AUDIO,OFERTEC PERU,NaN,Commission Credit Adjustment,NaN,2098633373,92.48,NaN,NaN,PEN,Reintegro a favor del seller por descuento CMR,NaN,NaN,Si,No,Si,Si,No,No,Si
671,NaN,16/10/2023,A Favor de Falabella,120800557,11780563,Alexis Ibañez,Comercial,AUDIO,OFERTEC PERU,NaN,Commission Adjustment,NaN,2100315405,5.00,NaN,NaN,PEN,Comisión aplicada al reintegro a favor de Fala...,NaN,NaN,Si,No,Si,Si,No,No,Si
672,NaN,16/10/2023,A Favor de Falabella,120800557,11692250,Alexis Ibañez,Comercial,AUDIO,OFERTEC PERU,NaN,Commission Adjustment,NaN,2099465875,5.00,NaN,NaN,PEN,Comisión aplicada al reintegro a favor de Fala...,NaN,NaN,Si,No,Si,Si,No,No,Si
673,NaN,16/10/2023,A Favor de Falabella,120800557,11595034,Alexis Ibañez,Comercial,AUDIO,OFERTEC PERU,NaN,Commission Adjustment,NaN,2098633373,5.00,NaN,NaN,PEN,Comisión aplicada al reintegro a favor de Fala...,NaN,NaN,Si,No,Si,Si,No,No,Si
674,NaN,16/10/2023,A Favor del Seller,119582223,11590196,Alexis Ibañez,Comercial,AUDIO,OFERTEC PERU,NaN,Commission Credit Adjustment,NaN,2098553363,20.00,NaN,NaN,PEN,Reintegro a favor del seller por descuento CMR,NaN,NaN,Si,No,Si,Si,No,No,Si
675,NaN,16/10/2023,A Favor de Falabella,119582223,11590196,Alexis Ibañez,Comercial,AUDIO,OFERTEC PERU,NaN,Commission Adjustment,NaN,2098553363,2.00,NaN,NaN,PEN,Comisión aplicada al reintegro a favor de Fala...,NaN,NaN,Si,No,Si,Si,No,No,Si
676,NaN,16/10/2023,A Favor del Seller,124238787,11691008,Alexis Ibañez,Comercial,AUDIO,OFERTEC PERU,NaN,Commission Credit Adjustment,NaN,2099453007,97.96,NaN,NaN,PEN,Reintegro a favor del seller por descuento CMR,NaN,NaN,Si,No,Si,Si,No,No,Si
677,NaN,16/10/2023,A Favor de Falabella,124238787,11691008,Alexis Ibañez,Comercial,AUDIO,OFERTEC PERU,NaN,Commission Adjustment,NaN,2099453007,5.00,NaN,NaN,PEN,Comisión aplicada al reintegro a favor de Fala...,NaN,NaN,Si,No,Si,Si,No,No,Si


In [786]:
# Generate metabase query
def generate_metabase_query(data1, data2, data3):
    orders_list1 = data1['Delivery Order Number'].drop_duplicates().to_list()
    orders_list2 = data2['Delivery Order Number'].drop_duplicates().to_list()
    orders_list3 = data3['Delivery Order Number'].drop_duplicates().to_list()

    orders_list = orders_list1 + orders_list2 + orders_list3

    sellers_list1 = data1['Seller ID*'].drop_duplicates().to_list()
    sellers_list2 = data2['Seller ID*'].drop_duplicates().to_list()
    sellers_list3 = data3['Seller ID*'].drop_duplicates().to_list()

    sellers_list = sellers_list1 + sellers_list2 + sellers_list3

    orders_list = ', '.join([f"'{order}'" for order in orders_list])
    sellers_list = ', '.join([f"'{seller}'" for seller in sellers_list])

    metabase_query = f"""
    SELECT
        seller_id,
        details ->> 'orderNumber' AS deliveryOrderNumber,
        details ->> 'deliveryOrderNumber' AS deliveryOrderNumber,
        details ->> 'productId' AS productId,
        details ->>'variantId' AS variantId,
        order_event_type AS ordereventtype,
        created_at as createdat,
        name,
        net_amount ->> 'currency' AS Moneda,
        net_amount ->> 'centAmount' AS Monto,
        net_amount ->> 'fraction' AS fraction,
        details ->> 'sellerOrderLineId' AS sellerOrderLineId
    FROM transaction
    WHERE tenant = '80ef7ae6-c10b-4684-8b2f-79d0bb4d91b5'
    AND details ->> 'deliveryOrderNumber' IN ({orders_list})
    AND seller_id IN ({sellers_list})
    """

    return print(metabase_query)

In [787]:
# Convert metabase data types
def convert_metabase_data_types(data):

    num_cols = ['monto','fraction']
    data[num_cols] = data[num_cols].apply(lambda x: pd.to_numeric(x.astype(str).str.replace(',', '.'), errors='coerce', downcast='float'))

    text_cols = ['seller_id','deliveryordernumber','sellerorderlineid','variantid']
    data[text_cols] = data[text_cols].apply(lambda x: x.astype(str).str.replace('(\.0$)', ''))

    date_cols = ['createdat']
    data[date_cols] = data[date_cols].apply(lambda x: pd.to_datetime(x, errors='coerce').dt.strftime('%d/%m/%Y'))

    return data

In [788]:
# Transform metabase data
def transform_metabase_data(data):
    devueltas = data.query("ordereventtype=='sellerReturnOrderLineUpdated'")

    ocs_devueltas = devueltas['deliveryordernumber'].drop_duplicates().to_list()
    sku_devueltas = devueltas['sellerorderlineid'].drop_duplicates().to_list()

    ajustadas = data.query("ordereventtype=='manualTransactionCreated'")

    ocs_ajustadas = ajustadas['deliveryordernumber'].drop_duplicates().to_list()
    sku_ajustadas = ajustadas['sellerorderlineid'].drop_duplicates().to_list()

    data = (
        data 
        .assign(**{
            'ocs_devueltas': lambda x: np.where((x['deliveryordernumber'].isin(ocs_devueltas)),'Si','No'),
            'sku_devueltas': lambda x: np.where((x['sellerorderlineid'].isin(sku_devueltas)),'Si','No'),
            'ocs_ajustadas': lambda x: np.where((x['deliveryordernumber'].isin(ocs_ajustadas)),'Si','No'),
            'sku_ajustadas': lambda x: np.where((x['sellerorderlineid'].isin(sku_ajustadas)),'Si','No'),
        }) 
        .assign(**{
            'validacion_dev': lambda x: np.where(((x['sku_devueltas']=='Si') | (x['sku_ajustadas']=='Si')),'No','Si')
        })
    )

    pivot1 = (
        data 
        .query("ocs_devueltas=='Si'") 
        .query("name.isin(['COMMISSION','COMMISSION_CREDIT'])")
        .pivot_table(
                index=['deliveryordernumber'],
                columns=['name'],
                values=['ocs_devueltas'],
                aggfunc=lambda x: (x == 'Si').sum()
            ) 
        ['ocs_devueltas']
        .reset_index()
        .rename(columns={'COMMISSION': 'total_skus','COMMISSION_CREDIT':'skus_devueltos'})
    )

    pivot2 = (
        data
        .query("ocs_ajustadas=='Si'") 
        .query("name.isin(['APPEALED_RETURN_CREDIT','COMMISSION_ADJUSTMENT','COMMISSION_CREDIT_ADJUSTMENT','FREE_SHIPPING_SELLER_COST_CREDIT_ADJUSTMENT','LOST_OR_DAMAGED_ORDER_ITEM_LEVEL_CREDIT'])") 
        .groupby(['deliveryordernumber'], dropna=False)
        .agg(total_skus=('sellerorderlineid','nunique'))
        .reset_index()

        #Falta transaccion APPEAL y FREE_SHIPPING_SELLER_COST # ver que tipo de ajuste se aplicó
    )

    return data, pivot1, pivot2

In [789]:
# Validate transactions with metabase data (2)
def metabase_validation(data, data2, data3):
    data = (
        data
        .merge(data2, left_on=['Delivery Order Number'], right_on=['deliveryordernumber'], how='left')
        .assign(**{'validacion_devolucion': lambda x: np.where(x['total_skus']>0,'No','Si')})
        .drop(['deliveryordernumber','total_skus','skus_devueltos'], axis=1)
        .merge(data3, left_on=['Delivery Order Number'], right_on=['deliveryordernumber'], how='left')
        .assign(**{'validacion_ajuste': lambda x: np.where(x['total_skus']>0,'No','Si')})
        .drop(['deliveryordernumber','total_skus'], axis=1)
    )

    return data

In [790]:
# Add seller order line id
def add_sellerorderlineid(data, metabase):
    sellerorderlineid = (
            metabase
            .loc[:, ['deliveryordernumber','variantid','sellerorderlineid']]
            .drop_duplicates()
            .assign(row = lambda x: x.groupby(['deliveryordernumber','variantid']).cumcount()+1)
            .rename(
                columns={
                    'deliveryordernumber': 'Delivery Order Number', 'variantid': 'Falabella SKU', 'sellerorderlineid': 'Seller Order Line Id*'
                }
            )
        )

    data = (
        data
        .assign(row = lambda x: x.groupby(['Delivery Order Number', 'Falabella SKU', 'Account Statement Group*']).cumcount() + 1)
        .drop(['Seller Order Line Id*'], axis=1)
        .merge(sellerorderlineid, how='left', on=['Delivery Order Number', 'Falabella SKU', 'row'])
        .loc[:, [
            'Revisar comentario', 'Fecha de envío', 'A Favor de', 'Falabella SKU',
            'Order Item Id', 'Solicitante ', 'Area', 'categoría',
            'Nombre del Seller en FSC', 'Seller ID*', 'Account Statement Group*',
            'Seller Order Line Id*', 'Delivery Order Number', 
            'Amount*', 'VAT', 'WHT', 'Currency*',
            'Comment', 'NÚMERO DE FACTURA', 'CENTRO DE COSTOS',
            'validacion_linea', 'validacion_seller_len', 'validacion_devolucion',
            'validacion_ajuste'
        ]]
    )

    return data

In [791]:
# Validate input seller (seller id + amount)
def input_validation(data, metabase):
    sellerid = (
            metabase
            .loc[:, ['seller_id','deliveryordernumber']]
            .drop_duplicates()
            .rename(columns={'deliveryordernumber': 'Delivery Order Number'})
    )

    amounts = (
            metabase
            .query("name=='COMMISSION'")
            .assign(**{'amount': lambda x: x['monto']/x['fraction']})
            .loc[:, ['sellerorderlineid','amount']]
            .rename(columns={'sellerorderlineid': 'Seller Order Line Id*'})
    )

    data = (
        data
        .merge(sellerid, how='left', on=['Delivery Order Number'])
        .assign(**{'seller_id': lambda x: np.where((x['seller_id'].isnull()), x['Seller ID*'], x['seller_id'])})
        .assign(**{'validacion_seller_id': lambda x: np.where((x['Seller ID*']==x['seller_id']),'Si','No')})
        .drop(['seller_id'], axis=1)
        .merge(amounts, how='left', on=['Seller Order Line Id*'])
        .assign(**{'ajuste_porc': lambda x: x['Amount*']/x['amount']})
        .assign(**{'validacion_monto_porc': lambda x: np.where((x['ajuste_porc']<0.75),'Si','No')})
        .assign(**{'validacion_monto_alto': lambda x: np.where((x['Amount*']<1000),'Si','No')})
        .drop(['amount','ajuste_porc'], axis=1)
    )

    return data

### **Data**

#### **1. Tipo de datos**

In [792]:
cmr = process_data_files(cmr_files, temp_dir).pipe(convert_data_types)
comisiones = process_data_files(comisiones_files, temp_dir).pipe(convert_data_types)
siniestros = process_data_files(siniestros_files, temp_dir).pipe(convert_data_types)

#### **2. Validaciones**

In [793]:
cmr = format_validation(cmr, validacion_linea=True, validacion_siniestro=False)
comisiones = format_validation(comisiones, validacion_linea=False, validacion_siniestro=False)
siniestros = format_validation(siniestros, validacion_linea=False, validacion_siniestro=True)

#### **3. Metabase query**

In [794]:
generate_metabase_query(cmr, comisiones, siniestros)


    SELECT
        seller_id,
        details ->> 'orderNumber' AS deliveryOrderNumber,
        details ->> 'deliveryOrderNumber' AS deliveryOrderNumber,
        details ->> 'productId' AS productId,
        details ->>'variantId' AS variantId,
        order_event_type AS ordereventtype,
        created_at as createdat,
        name,
        net_amount ->> 'currency' AS Moneda,
        net_amount ->> 'centAmount' AS Monto,
        net_amount ->> 'fraction' AS fraction,
        details ->> 'sellerOrderLineId' AS sellerOrderLineId
    FROM transaction
    WHERE tenant = '80ef7ae6-c10b-4684-8b2f-79d0bb4d91b5'
    AND details ->> 'deliveryOrderNumber' IN ('2095913098', '2095917783', '2095872223', '2095849451', '2095529444', '2095862853', '2095740648', '2095521074', '2111949347', '2111180541', '2111173112', '2111058049', '2110986157', '2110933933', '2110913347', '2110792216', '2110779794', '2110759627', '2110672027', '2110646477', '2110628680', '2110604987', '2110605001', '2110581914', '21

#### **4. Metabase data**

In [795]:
metabase_files = glob.glob(os.path.join(ruta_ajustes, "query*.xlsx"))

metabase = (
            process_data_files(metabase_files, temp_dir)
            .pipe(convert_metabase_data_types)
)

metabase, pivot_devueltas, pivot_ajustadas = transform_metabase_data(metabase)

#### **5. Metabase validation**

In [796]:
cmr = (
      metabase_validation(cmr, pivot_devueltas, pivot_ajustadas)
      .pipe(add_sellerorderlineid, metabase)
      .pipe(input_validation, metabase)
)

siniestros = (
      metabase_validation(siniestros, pivot_devueltas, pivot_ajustadas)
      .pipe(add_sellerorderlineid, metabase)
      .pipe(input_validation, metabase)
)

comisiones = (
      metabase_validation(comisiones, pivot_devueltas, pivot_ajustadas)
      .pipe(add_sellerorderlineid, metabase)
      .pipe(input_validation, metabase)
)

In [797]:
# Aqui termina CMR (25/10/23 17:22 - 17:24) FIN" y continua!! # 13min7s
# HASTA AQUI OK!

In [800]:
siniestros.query("`Delivery Order Number`=='2072271588'")

,Revisar comentario,Fecha de envío,A Favor de,Falabella SKU,Order Item Id,Solicitante,Area,categoría,Nombre del Seller en FSC,Seller ID*,Account Statement Group*,Seller Order Line Id*,Delivery Order Number,Amount*,VAT,WHT,Currency*,Comment,NÚMERO DE FACTURA,CENTRO DE COSTOS,validacion_linea,validacion_seller_len,validacion_devolucion,validacion_ajuste,validacion_seller_id,validacion_monto_porc,validacion_monto_alto
19,NaN,20/10/2023,A favor del seller,114087887,nan,Melissa Huerta,Seller experience,NaN,marketcellperu,SC7F9F0,Lost or Damaged (Order Item Level) Credit,4b5cb996-9579-4cc0-874a-07a895000479,2072271588,3265.08,NaN,NaN,PEN,NaN,E001-878,Loc HD C&C OOLL,Si,Si,Si,No,Si,No,No


### **2. Comisiones**

In [ ]:
#comisiones + siniestros??

,Revisar comentario,Fecha de envío,A Favor de,Falabella SKU,Order Item Id,Solicitante,Area,categoría,Nombre del Seller en FSC,Seller ID*,Account Statement Group*,Seller Order Line Id*,Delivery Order Number,Amount*,VAT,WHT,Currency*,Comment,NÚMERO DE FACTURA,CENTRO DE COSTOS,File,validacion_seller_len
0,NaN,20/10/2023,A favor del seller,116454174,nan,Melissa Huerta,Seller experience,NaN,Maxicompragsc-Sceb6845c8382a8,SCEB684,Lost or Damaged (Order Item Level) Credit,NaN,2074865703,12.69,NaN,NaN,PEN,NaN,E001-610,Loc HD Reparto a Clientes OOLL,Plantilla de Ajustes F.COM 20.10.2023_Melissa ...,Si
1,NaN,20/10/2023,A favor del seller,116454174,nan,Melissa Huerta,Seller experience,NaN,Maxicompragsc-Sceb6845c8382a8,SCEB684,Lost or Damaged (Order Item Level) Credit,NaN,2074865703,12.69,NaN,NaN,PEN,NaN,E001-610,Loc HD Reparto a Clientes OOLL,Plantilla de Ajustes F.COM 20.10.2023_Melissa ...,Si
2,NaN,20/10/2023,A favor del seller,121289713,nan,Melissa Huerta,Seller experience,NaN,Maxicompragsc-Sceb6845c8382a8,SCEB684,Lost or Damaged (Order Item Level) Credit,NaN,2074865703,43.12,NaN,NaN,PEN,NaN,E001-610,Loc HD Reparto a Clientes OOLL,Plantilla de Ajustes F.COM 20.10.2023_Melissa ...,Si
3,NaN,20/10/2023,A favor del seller,118269612,nan,Melissa Huerta,Seller experience,NaN,Inroplasgsc-Scbf87433029338,SCBF874,Lost or Damaged (Order Item Level) Credit,NaN,2085984401,40.19,NaN,NaN,PEN,NaN,E001-7257,Loc HD Reparto a Clientes OOLL,Plantilla de Ajustes F.COM 20.10.2023_Melissa ...,Si
4,NaN,20/10/2023,A favor del seller,118264650,nan,Melissa Huerta,Seller experience,NaN,Inroplasgsc-Scbf87433029338,SCBF874,Lost or Damaged (Order Item Level) Credit,NaN,2085984401,4.00,NaN,NaN,PEN,NaN,E001-7257,Loc HD Reparto a Clientes OOLL,Plantilla de Ajustes F.COM 20.10.2023_Melissa ...,Si
5,NaN,20/10/2023,A favor del seller,118264650,nan,Melissa Huerta,Seller experience,NaN,Inroplasgsc-Scbf87433029338,SCBF874,Lost or Damaged (Order Item Level) Credit,NaN,2085984401,4.00,NaN,NaN,PEN,NaN,E001-7257,Loc HD Reparto a Clientes OOLL,Plantilla de Ajustes F.COM 20.10.2023_Melissa ...,Si
6,NaN,20/10/2023,A favor del seller,118065782,nan,Melissa Huerta,Seller experience,NaN,TIENDAS MADI,SCDB5AF,Lost or Damaged (Order Item Level) Credit,NaN,2056650977,242.10,NaN,NaN,PEN,NaN,E001-235,Local Home Delivery Tren Logístico,Plantilla de Ajustes F.COM 20.10.2023_Melissa ...,Si
7,NaN,20/10/2023,A favor del seller,118065782,nan,Melissa Huerta,Seller experience,NaN,TIENDAS MADI,SCDB5AF,Lost or Damaged (Order Item Level) Credit,NaN,2056650977,242.10,NaN,NaN,PEN,NaN,E001-235,Local Home Delivery Tren Logístico,Plantilla de Ajustes F.COM 20.10.2023_Melissa ...,Si
8,NaN,20/10/2023,A favor del seller,116247407,nan,Melissa Huerta,Seller experience,NaN,Ebestpricegsc-Scdc69d09042cc6,SCDC69D,Lost or Damaged (Order Item Level) Credit,NaN,2089738116,23.78,NaN,NaN,PEN,NaN,E001-1166,Loc HD Reparto a Clientes OOLL,Plantilla de Ajustes F.COM 20.10.2023_Melissa ...,Si
9,NaN,20/10/2023,A favor del seller,120065433,nan,Melissa Huerta,Seller experience,NaN,CHIKI LOVE,SCD062D,Lost or Damaged (Order Item Level) Credit,NaN,2082644199,29.64,NaN,NaN,PEN,NaN,EB01-143,CC Loc HD 3P FBy Ops Post Vent,Plantilla de Ajustes F.COM 20.10.2023_Melissa ...,Si


In [ ]:
# se hace las mismas validaciones que CMR

# 3. Siniestros

In [ ]:
# aqui la transaccion siempre va ser "Lost Or Damaged"

#### **1. C**